In [275]:
class Pessoa:
    def __init__(self, strName, intAge):
        self.name = strName
        self.age = intAge

    def __str__(self):
        return "Nome: "+self.name+" Idade: "+str(self.age)
    
    def smaller(self, value):
        return True if value == None else (self.age < value.age)
    
    def bigger(self, value):
        return True if value == None else (self.age > value.age)

In [283]:
class TreeNode:
    def __init__(self, value):
        self.__info:Pessoa = value
        self.__left:TreeNode = None
        self.__right:TreeNode = None
        self.__father:TreeNode = None
        self.__num:int = 1
    
    @property
    def info(self):
        return self.__info

    @info.setter
    def info(self,value):
        self.__info = value

    @property
    def right(self):
        return self.__right
    
    @right.setter
    def right(self,value):
        self.__right = value

    @property
    def left(self):
        return self.__left
    
    @left.setter
    def left(self,value):
        self.__left = value
    
    @property
    def father(self):
        return self.__father
    
    @father.setter
    def father(self,value):
        self.__father = value
 
    @property
    def num(self):
        return self.__num
    
    @num.setter
    def num(self,value):
        self.__num = value
    
    
    def setRight(self, node):
        node.num = 2*self.num+1
        self.right = self.__insertNode(node)

    def setLeft(self, node):
        node.num = 2*self.num
        self.left = self.__insertNode(node)

    def isRoot(self):
        return self.father == None
    
    def isLeaf(self):
        return self.left == None and self.right == None

    def isLeft(self):
        return not self.isRoot() and self.father.left == self  
    
    def isRight(self):
        return not self.isRoot() and self.father.right == self
    
    def __insertNode(self, node):
        node.father = self
        return node
    
    def delete(self):
        if self.isRight() :
           self.father.right = None
        elif self.isLeft():
           self.father.left = None
        del self

    @property
    def brother(self):
        if(not self.isRoot()):
            if (self.isLeft()):
                return self.father.right
            else:
                return self.father.left
    @staticmethod
    def exchangeInfo(node1, node2):
        temp = node1.info
        node1.info = node2.info
        node2.info = temp
        return [node1,node2]
    
    def smaller(self, node:TreeNode):
        return self if node == None or self.info.smaller(node.info) else node

In [317]:
from cmath import inf
from turtle import right


class TreeHeap:
    def __init__(self, value=None, type="MIN"):
        self.__length:int = 0 if value == None else 1
        self.__root:TreeNode = value 
        self.__last:TreeNode = value
        self.__type:str = type
    
    @property
    def length(self):
        return self.__length
    
    @property
    def root(self):
        return self.__root
    
    @property
    def last(self):
        return self.__last
    
    @property
    def type(self):
        return self.__type
    
    # recebe o número do nó (node_num) e retorna uma lista de binários
    def __pathNode(self, node_num):
        path = [int(x) for x in bin(node_num)[2:]]
        return path[1:]

    # essa função já insere um nó (new_node) na posição correta do Heap
    # ou seja, na posição que equivale à quantidade de elementos mais o novo
    def __insertLast(self, new_node):
        bin_path = self.__pathNode(self.length+1)
        node = self.root
        for x in bin_path[:-1]:
            node = node.left if (x == 0) else node.right
        if(bin_path[-1]==0):
            node.setLeft(new_node)
        else:
            node.setRight(new_node)
    
    # adaptar essa função para usar o caminho retornado por self.__pathNode(numero do nó)
    # para percorrer a árvore a partir da raiz:
    # se 0 vai para o filho esquerdo, se 1, vai para o filho a direita
    def insert(self, value):
        new_node = TreeNode(value)
        if(self.__root == None):
            self.__root = new_node
            print(f"Inserindo {new_node.info.name}:{new_node.info.age} na raiz")
        else:
            node = self.__last
            if self.__last.isLeft() :
                node.father.setRight(new_node)
                print(f"Inserindo {new_node.info.name}:{new_node.info.age} como filho direito de {node.father.info.name}")
            else:
                while(not node.isRoot() and node.isRight()):
                    node = node.father
                if not node.isRoot() :
                    node = node.brother
                while (not node.isLeaf()):
                    node = node.left
                node.setLeft(new_node)
                print(f"Inserindo {new_node.info.name}:{new_node.info.age} como filho esquerdo de {node.info.name}")
        self.__length += 1
        self.__last = new_node
        print(f"Último {self.__last.info.name}:{self.__last.info.age}")
        self.upHeap()
        print(f"Após upHeap, último {self.__last.info.name}:{self.__last.info.age}")
        
    def remove(self):
        if self.__root != None :
            root_info = self.__root.info
            if not self.__last.isRoot() :
                self.__root, self.__last = TreeNode.exchangeInfo(self.__root, self.__last)
                node = self.__last
                while(node != self.__root and node.isLeft()):
                    node = node.father
                if node.isRight() :
                    node = node.brother
                while node.right != None :
                    node = node.right
                self.__last.delete()
                self.__last = node
                self.downHeap()
            else:
                self.__root = None
            self.__length -= 1
        else:
            raise ValueError('Empty Heap!')
        
        return root_info
    
    def downHeap(self):
        if not self.__root.isLeaf() :
            father = self.__root
            flag = True
            if self.__type == "MIN":
                son = father.left.smaller(father.right)
                while flag and son.info.smaller(father.info) :
                    son,father = TreeNode.exchangeInfo(son,father)
                    father = son
                    if father.left != None :
                        son = father.left.smaller(father.right)
                    else:
                        flag = False
            elif self.__type == "MAX":
                if son.info.bigger(father.info) :
                    son,father = TreeNode.exchangeInfo(son,father)
                    self.__last = son
                    son = father
                    father = son.father
                while(father != None and son.info.bigger(father.info)):
                    son,father = TreeNode.exchangeInfo(son,father)
                    son = father
                    father = son.father
            else:
                raise ValueError('Wrong Heap type. It can be MIN or MAX')
              
    def upHeap(self):
        if not self.__last.isRoot() :
            son = self.__last
            parent = self.__last.father
            if self.__type == "MIN":
                if son.info.smaller(parent.info) :
                    son,parent = TreeNode.exchangeInfo(son,parent)
                    self.__last = son
                    son = parent
                    parent = son.father
                while(parent != None and son.info.smaller(parent.info)):
                    son,parent = TreeNode.exchangeInfo(son,parent)
                    son = parent
                    parent = son.father
            elif self.__type == "MAX":
                if son.info.bigger(parent.info) :
                    son,parent = TreeNode.exchangeInfo(son,parent)
                    self.__last = son
                    son = parent
                    parent = son.father
                while(parent != None and son.info.bigger(parent.info)):
                    son,parent = TreeNode.exchangeInfo(son,parent)
                    son = parent
                    parent = son.father
            else:
                raise ValueError('Wrong Heap type. It can be MIN or MAX')
            
    def __recursiveStr(self,node,str_final):
        str_final += f"{node.num} {str(node.info)}\n"
        if node.left != None :
            str_final = self.__recursiveStr(node.left,str_final)
        if node.right != None :
            str_final = self.__recursiveStr(node.right,str_final)
        
        return str_final
    
    def __recursiveList(self,node,list_final):
        list_final.append([node.num,node.info.name,node.info.age])
        if node.left != None :
            list_final = self.__recursiveList(node.left,list_final)
        if node.right != None :
            list_final = self.__recursiveList(node.right,list_final)
        
        return list_final
    
    def toList(self):
        if self.__root != None :
            return self.__recursiveList(self.__root,[])
        else:
            return ["Empty Heap"]
    
    def printGraph(self):
        heap_list = self.toList()
        heap_list.sort()
        values = []
        for x in heap_list:
            values.append(f"{x[1]}:{x[2]}")
        root = build(values)
        print(root)
        
    def __str__(self) -> str:
        if self.__root != None :
            heap_list = self.toList()
            heap_list.sort()
            str_list = ""
            for x in heap_list:
                str_list += f"{x[0]} - {x[1]}:{x[2]}\n"
            return str_list
        else:
            return "Empty Heap"

In [319]:
from binarytree import build

andre = Pessoa("Andre",25)
marcos = Pessoa("Marcos",30)
maria = Pessoa("Maria",28)
joao = Pessoa("João",15)
helder = Pessoa("Hélder",35)
vana = Pessoa("Vana",38)
nicole = Pessoa("Nicole",5)
otavio = Pessoa("Otavio",65)
claudice = Pessoa("Claudice", 63)


heap = TreeHeap(type="MIN")

print("Inserindo elementos no HEAP")
heap.insert(andre)
heap.printGraph()

heap.insert(marcos)
heap.printGraph()

heap.insert(helder)
heap.printGraph()

heap.insert(vana)
heap.printGraph()

heap.insert(nicole)
heap.printGraph()

heap.insert(maria)
heap.printGraph()

heap.insert(joao)
heap.printGraph()

heap.insert(otavio)
heap.printGraph()

heap.insert(claudice)
heap.printGraph()

print("Removendo elementos do HEAP")
for x in range(9):
    print(f"Nó removido: {heap.remove()}")
    heap.printGraph()
    
print("Inserindo elementos no HEAP")
heap.insert(claudice)
heap.printGraph()

heap.insert(otavio)
heap.printGraph()

heap.insert(joao)
heap.printGraph()

heap.insert(maria)
heap.printGraph()

heap.insert(nicole)
heap.printGraph()

heap.insert(vana)
heap.printGraph()

heap.insert(helder)
heap.printGraph()

heap.insert(marcos)
heap.printGraph()

heap.insert(andre)
heap.printGraph()

print("Removendo elementos do HEAP")
for x in range(1):
    print(f"Nó removido: {heap.remove()}")
    heap.printGraph()


Inserindo elementos no HEAP
Inserindo Andre:25 na raiz
Último Andre:25
Após upHeap, último Andre:25

Andre:25

Inserindo Marcos:30 como filho esquerdo de Andre
Último Marcos:30
Após upHeap, último Marcos:30

      ____Andre:25
     /
Marcos:30

Inserindo Hélder:35 como filho direito de Andre
Último Hélder:35
Após upHeap, último Hélder:35

      ____Andre:25____
     /                \
Marcos:30          Hélder:35

Inserindo Vana:38 como filho esquerdo de Marcos
Último Vana:38
Após upHeap, último Vana:38

              ____Andre:25____
             /                \
     ___Marcos:30          Hélder:35
    /
Vana:38

Inserindo Nicole:5 como filho direito de Marcos
Último Nicole:5
Após upHeap, último Marcos:30

             ______________Nicole:5____
            /                          \
     ___Andre:25____                Hélder:35
    /               \
Vana:38          Marcos:30

Inserindo Maria:28 como filho esquerdo de Hélder
Último Maria:28
Após upHeap, último Hélder:35

       